# Lab 3B: きめ細かいアクセス制御を持つ修復エージェント

## 概要
このラボは、JWT 認証と Lambda インターセプターを使用したロールベースのアクセス制御を導入することで、Lab 3A を拡張します。SRE ユーザーは修復を計画できますが、承認者のみが実行できるという職務分離パターンを実装します。

## 学習目標
- Cognito を使用した JWT ベース認証を理解する
- Lambda インターセプターによるロールベースアクセス制御（RBAC）を実装する
- カスタム認可を持つ AgentCore ゲートウェイをデプロイおよび構成する
- 自動化された修復ワークフローで職務分離を強制する
- アクセス決定を監視および監査する

## ペルソナとアクセスモデル

### SRE ロール
- **権限:** 修復計画の作成と検証
- **制限:** 修復を実行できない
- **ユースケース:** 問題を診断し、修正を提案

### 承認者ロール
- **権限:** 修復の実行と検証
- **制限:** なし（フルアクセス）
- **ユースケース:** SRE 計画をレビューし、承認された修正を実行

---

# ステップ 1: ユースケースの説明

## シナリオ

組織ではインフラストラクチャ変更に対する**職務分離**が必要です：

- **SRE チーム:** 問題を診断し、修復計画を作成
- **承認者チーム:** 承認された変更をレビューおよび実行

**問題:** アクセス制御がなければ、誰でもインフラストラクチャ変更を実行でき、コンプライアンスとセキュリティのリスクが発生します。

**解決策:** JWT 認証 + Lambda インターセプターを使用して、ゲートウェイレベルでロールベースの権限を強制します。

## なぜ JWT + Lambda インターセプターなのか？

**JWT 認証:**
- 業界標準のトークン形式
- ユーザー ID とグループメンバーシップを含む
- Cognito によって暗号的に署名
- データベース検索が不要

**Lambda インターセプター:**
- ゲートウェイの REQUEST フェーズで実行
- ランタイムにルーティングする前に JWT クレームを検査
- きめ細かいアクセス制御を強制
- CloudWatch ログによる監査証跡を提供

**メリット:**
- 認可ロジックの集中化
- エージェントコードの変更不要
- スケーラブルでサーバーレス
- 監査と変更が容易

## ワークフロー図

```
┌─────────────────────────────────────────────────────────────────┐
│                    SRE ワークフロー（計画）                      │
└─────────────────────────────────────────────────────────────────┘

SRE ユーザー → Cognito 認証 → JWT (groups: ["sre"])
                                ↓
                          Gateway がトークンを受信
                                ↓
                    Lambda インターセプターがチェック:
                    - cognito:groups = ["sre"] を抽出
                    - ツール: generate_remediation_plan
                    - 権限: ✅ 許可
                                ↓
                          Runtime が実行
                                ↓
                    修復計画が作成 ✅

┌─────────────────────────────────────────────────────────────────┐
│              SRE ワークフロー（実行の試行）                      │
└─────────────────────────────────────────────────────────────────┘

SRE ユーザー → Cognito 認証 → JWT (groups: ["sre"])
                                ↓
                          Gateway がトークンを受信
                                ↓
                    Lambda インターセプターがチェック:
                    - cognito:groups = ["sre"] を抽出
                    - ツール: execute_remediation_step
                    - 権限: ❌ 拒否
                                ↓
                    リクエストが Gateway でブロック ❌

┌─────────────────────────────────────────────────────────────────┐
│                 承認者ワークフロー（実行）                       │
└─────────────────────────────────────────────────────────────────┘

承認者 → Cognito 認証 → JWT (groups: ["approvers"])
                                ↓
                          Gateway がトークンを受信
                                ↓
                    Lambda インターセプターがチェック:
                    - cognito:groups = ["approvers"] を抽出
                    - ツール: execute_remediation_step
                    - 権限: ✅ 許可
                                ↓
                          Runtime が実行
                                ↓
                    修復が実行 ✅
```

In [ ]:
%pip install -q -r requirements.txt
print("✅ Workshop dependencies installed")

In [ ]:
# Verify Lab 3A completion
from lab_helpers.parameter_store import get_parameter
from lab_helpers.constants import PARAMETER_PATHS

try:
    user_pool_id = get_parameter(PARAMETER_PATHS['cognito']['user_pool_id'])
    print("✅ Lab 3A prerequisites verified")
    print(f"   Cognito User Pool: {user_pool_id}")
    print("   Ready to proceed with Lab 3B")
except Exception as e:
    print("❌ Lab 3A not complete. Please run Lab-03a-remediation-agent.ipynb first.")
    raise

---

# ステップ 2: Cognito トークンとグループクレーム

## 目標
Cognito トークンに含まれるものと、グループクレームがアクセス制御を強制する方法を理解する。

## アクティビティ
- [ ] SRE ユーザーの Cognito トークンを取得
- [ ] 承認者ユーザーの Cognito トークンを取得
- [ ] JWT トークンをデコード
- [ ] 両方のトークンからグループクレームを表示
- [ ] トークンペイロード間の違いを比較

In [ ]:
# Imports
import json
import boto3
from datetime import datetime
from lab_helpers.parameter_store import get_parameter
from lab_helpers.constants import PARAMETER_PATHS
from lab_helpers.config import AWS_REGION
from lab_helpers.lab_03 import decode_jwt, print_token_claims, compare_tokens

cognito = boto3.client('cognito-idp', region_name=AWS_REGION)

### SRE ユーザートークン

In [ ]:
# Authenticate SRE user
user_client_id = get_parameter(PARAMETER_PATHS['cognito']['user_auth_client_id'])
sre_username = get_parameter(PARAMETER_PATHS['cognito']['test_user_email'])
sre_password = get_parameter(PARAMETER_PATHS['cognito']['test_user_password'])

sre_response = cognito.initiate_auth(
    ClientId=user_client_id,
    AuthFlow='USER_PASSWORD_AUTH',
    AuthParameters={'USERNAME': sre_username, 'PASSWORD': sre_password}
)

sre_token = sre_response['AuthenticationResult']['AccessToken']
print(f"✅ SRE authenticated: {sre_username}")
print(f"   Token (first 50 chars): {sre_token}")

In [ ]:
# Decode and display SRE token
sre_claims = decode_jwt(sre_token)
print_token_claims(sre_claims, "SRE Token Claims")

### 承認者ユーザートークン

In [ ]:
# Authenticate Approver user
approver_username = get_parameter(PARAMETER_PATHS['cognito']['approver_user_email'])
approver_password = get_parameter(PARAMETER_PATHS['cognito']['approver_user_password'])

approver_response = cognito.initiate_auth(
    ClientId=user_client_id,
    AuthFlow='USER_PASSWORD_AUTH',
    AuthParameters={'USERNAME': approver_username, 'PASSWORD': approver_password}
)

approver_token = approver_response['AuthenticationResult']['AccessToken']
print(f"✅ Approver authenticated: {approver_username}")
print(f"   Token (first 50 chars): {approver_token[:50]}...")

In [ ]:
# Decode and display Approver token
approver_claims = decode_jwt(approver_token)
print_token_claims(approver_claims, "Approver Token Claims")

### トークンペイロードの比較

In [ ]:
# Compare tokens side-by-side
compare_tokens(sre_claims, approver_claims)

### 主要な概念

**JWT 構造:**
- `header.payload.signature`
- ペイロードにはクレーム（ユーザー属性）が含まれる
- 署名がトークンの真正性を検証

**Cognito のグループクレーム:**
- `cognito:groups` はトークンに自動的に含まれる
- ユーザーが属するすべてのグループをリスト
- カスタム属性は不要

**認可フロー:**
```
ユーザー → Cognito → cognito:groups を含む JWT
                        ↓
                  Gateway がトークンを受信
                        ↓
                Lambda インターセプターが cognito:groups を抽出
                        ↓
                グループを許可されたツールにマッピング
                        ↓
                許可/拒否の決定
```

**権限マッピング（Lambda で実装予定）:**
```python
GROUP_PERMISSIONS = {
    "sre": ["generate_remediation_plan"],
    "approvers": ["execute_remediation_step", "validate_remediation_environment"]
}
```

---

# ステップ 3: インターセプター Lambda のデプロイ

## 目標
ゲートウェイレベルでアクセス制御を強制する Lambda 関数を作成してデプロイする。

## インターセプターロジック
Lambda は受信リクエストを検査し、以下を行います：
- [ ] リクエストヘッダーから JWT を抽出
- [ ] JWT 署名を検証
- [ ] グループクレームをチェック
- [ ] アクションベースのアクセス制御を強制
- [ ] 許可/拒否の決定を返却

### インターセプターコードのレビュー

In [ ]:
# Review interceptor Lambda code
with open('lab_helpers/lab_03/interceptor-request.py', 'r') as f:
    print(f.read())

### Lambda 関数のデプロイ

In [ ]:
# Deploy interceptor Lambda in us-west-2
from lab_helpers.lab_03 import deploy_interceptor
from lab_helpers.parameter_store import put_parameter
from lab_helpers.constants import PARAMETER_PATHS
from lab_helpers.config import AWS_REGION, WORKSHOP_NAME

print("📦 Deploying interceptor Lambda...")
function_arn = deploy_interceptor(region=AWS_REGION, prefix=WORKSHOP_NAME)

# Store for later use
put_parameter(PARAMETER_PATHS['lab_03b']['interceptor_function_arn'], function_arn)

print(f"\n✅ Interceptor ready: {function_arn}")


**注意:** インターセプターはステップ 8-10 で、異なるユーザートークンでゲートウェイを呼び出すことでテストします。

---

# ステップ 4: Lab 3A Gateway のクリーンアップ

## 目標
Lab 3A のゲートウェイを削除します。きめ細かいアクセス制御に必要な Lambda インターセプター設定がないためです。

## なぜゲートウェイを削除するのか？

**Lab 3A ゲートウェイ設定:**
- 認証不要
- インターセプター未接続
- すべてのツールへのオープンアクセス

**Lab 3B ゲートウェイ要件:**
- Cognito による JWT 認証
- REQUEST フェーズでの Lambda インターセプター
- ロールベースのアクセス制御の強制

**なぜ更新しないのか？**
ゲートウェイインターセプター設定は作成後に変更できません—初期デプロイ時に設定する必要があります。したがって、以下が必要です：
1. Lab 3A ゲートウェイを削除
2. インターセプター設定を持つ新しいゲートウェイを作成
3. 既存のランタイムを再利用（変更不要）

**削除されるもの:**
- ✅ ゲートウェイ（インターセプター設定が必要）
- ✅ ゲートウェイターゲット（ゲートウェイと共に自動削除）

**再利用されるもの:**
- ✅ ランタイム（同じエージェントロジック、異なるゲートウェイ）
- ✅ Cognito ユーザープール（既に設定済み）
- ✅ Lambda インターセプター（ステップ 3 でデプロイ済み）

### Lab 3A ゲートウェイの削除

In [ ]:
import json
import boto3
import time

# Initialize bedrock-agentcore-control client
agentcore_client = boto3.client('bedrock-agentcore-control', region_name=AWS_REGION)

# List all gateways
print("🔍 Listing all gateways...")
response = agentcore_client.list_gateways()
gateways = response.get('items', [])

if not gateways:
    print("ℹ️  No gateways found")
else:
    print(f"📋 Found {len(gateways)} gateway(s)\n")
    
    # Filter to only Lab 3A gateways (to be replaced by Lab 3B)
    lab_3a_gateways = [gw for gw in gateways if 'aiml301-remediation-gateway' in gw.get('name', '')]
    
    if not lab_3a_gateways:
        print("ℹ️  No Lab 3A gateways to delete")
    else:
        print(f"🗑️  Deleting {len(lab_3a_gateways)} Lab 3A gateway(s)\n")
        
        # Delete each Lab 3A gateway
        for gateway in lab_3a_gateways:
            gateway_id = gateway['gatewayId']
            gateway_name = gateway.get('name', 'N/A')
            
            print(f"🗑️  Deleting: {gateway_name} ({gateway_id})")
            
            try:
                # First, list and delete all targets
                targets_response = agentcore_client.list_gateway_targets(gatewayIdentifier=gateway_id)
                targets = targets_response.get('items', [])
                
                for target in targets:
                    target_id = target['targetId']
                    print(f"   🎯 Deleting target: {target_id}")
                    agentcore_client.delete_gateway_target(
                        gatewayIdentifier=gateway_id,
                        targetId=target_id
                    )
                
                # Wait for all targets to be deleted
                if targets:
                    print(f"   ⏳ Waiting for targets to be deleted...")
                    for _ in range(30):
                        time.sleep(2)
                        check = agentcore_client.list_gateway_targets(gatewayIdentifier=gateway_id)
                        if len(check.get('items', [])) == 0:
                            break
                
                # Now delete the gateway
                agentcore_client.delete_gateway(gatewayIdentifier=gateway_id)
                print(f"   ✅ Deleted")
                
            except Exception as e:
                print(f"   ❌ Error: {e}")
        
        print(f"\n✅ Cleanup complete")


**注意:** ランタイムは Lab 3A から再利用します—削除は不要です。

---

# ステップ 5: JWT 認証を持つ新しいゲートウェイとターゲットの作成

## 目標
JWT 認証と Lambda インターセプターで設定された新しいゲートウェイをデプロイする。

### ゲートウェイ設定

**ゲートウェイ詳細:**
- 名前: `interceptor-gateway-jwt-[random]`
- プロトコル: MCP
- オーソライザータイプ: CUSTOM_JWT
- インターセプトポイント: REQUEST

**JWT 設定:**
```json
{
  "authorizerType": "CUSTOM_JWT",
  "discoveryUrl": "https://cognito-idp.us-west-2.amazonaws.com/us-west-2_POOL_ID/.well-known/openid-configuration",
  "allowedClients": [
    "CLIENT_ID_1",
    "CLIENT_ID_2"
  ]
}
```

**インターセプター設定:**
```json
{
  "interceptionPoints": ["REQUEST"],
  "interceptor": {
    "lambda": {
      "arn": "arn:aws:lambda:us-west-2:ACCOUNT:function:custom-interceptor-request"
    }
  },
  "inputConfiguration": {
    "passRequestHeaders": true
  }
}
```

### ゲートウェイの作成

In [ ]:
import random
import string
import time
import boto3
from lab_helpers.parameter_store import get_parameter, put_parameter
from lab_helpers.constants import PARAMETER_PATHS
from lab_helpers.config import AWS_REGION
from lab_helpers.lab_03.gateway_setup import AgentCoreGatewaySetup

# Initialize bedrock-agentcore-control client
agentcore_client = boto3.client('bedrock-agentcore-control', region_name= AWS_REGION)

# Get configuration values
user_pool_id = get_parameter(PARAMETER_PATHS['cognito']['user_pool_id'])
user_client_id = get_parameter(PARAMETER_PATHS['cognito']['user_auth_client_id'])
m2m_client_id = get_parameter(PARAMETER_PATHS['cognito']['m2m_client_id'])
interceptor_arn = get_parameter(PARAMETER_PATHS['lab_03b']['interceptor_function_arn'])

# Create Gateway IAM role in us-west-2
gateway_setup = AgentCoreGatewaySetup(region=AWS_REGION, prefix=WORKSHOP_NAME, verbose=False)
role_info = gateway_setup.create_gateway_service_role()
role_arn = role_info['role_arn']
print(f"✅ Gateway role: {role_arn}")

# Generate unique gateway name
suffix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=10))
gateway_name = f"aiml301-interceptor-gateway-jwt-{suffix}"

print(f"📤 Creating gateway: {gateway_name}")
print(f"   Gateway region: us-west-2")
print(f"   Cognito region: {AWS_REGION}")
print(f"   Role: {role_arn}")
print(f"   Interceptor: {interceptor_arn}")

# Create gateway
response = agentcore_client.create_gateway(
    name=gateway_name,
    protocolType="MCP",
    protocolConfiguration={
        "mcp": {
            "supportedVersions": ["2025-03-26"]
        }
    },
    authorizerType="CUSTOM_JWT",
    authorizerConfiguration={
        "customJWTAuthorizer": {
            "discoveryUrl": f"https://cognito-idp.{AWS_REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration",
            "allowedClients": [user_client_id, m2m_client_id]
        }
    },
    interceptorConfigurations=[
        {
            "interceptionPoints": ["REQUEST"],
            "interceptor": {
                "lambda": {"arn": interceptor_arn}
            },
            "inputConfiguration": {
                "passRequestHeaders": True
            }
        }
    ],
    roleArn=role_arn
)

gateway_id = response['gatewayId']
gateway_arn = response['gatewayArn']

print(f"\n✅ Gateway created: {gateway_id}")
print(f"   ARN: {gateway_arn}")


In [ ]:
# Wait for gateway to be READY
print("⏳ Waiting for gateway to be ready...")
while True:
    response = agentcore_client.get_gateway(gatewayIdentifier=gateway_id)
    
    status = response['status']
    if status == 'READY':
        gateway_url = response['gatewayUrl']
        print(f"✅ Gateway ready: {status}")
        print(f"   URL: {gateway_url}")
        break
    elif status == 'FAILED':
        print(f"❌ Gateway creation failed")
        break
    print(f"   Status: {status}")
    time.sleep(5)

# Store for later use
put_parameter(PARAMETER_PATHS['lab_03b']['gateway_id'], gateway_id)
put_parameter(PARAMETER_PATHS['lab_03b']['gateway_url'], gateway_url)


### ゲートウェイ設定の確認

In [ ]:
# Display full gateway configuration
result = agentcore_client.get_gateway(gatewayIdentifier=gateway_id)

print("📋 Gateway Configuration:")
print("=" * 70)
print(json.dumps(result, indent=2, default=str))
print("=" * 70)

# Highlight key configuration elements
print("\n🔑 Key Configuration Elements:")
print(f"   Gateway ID: {result['gatewayId']}")
print(f"   Gateway URL: {result['gatewayUrl']}")
print(f"   Protocol: {result['protocolType']}")
print(f"   Authorizer: {result['authorizerType']}")

# JWT Configuration
jwt_config = result['authorizerConfiguration']['customJWTAuthorizer']
print(f"\n🔐 JWT Configuration:")
print(f"   Discovery URL: {jwt_config['discoveryUrl']}")
print(f"   Allowed Clients: {len(jwt_config['allowedClients'])} clients")
for i, client in enumerate(jwt_config['allowedClients'], 1):
    print(f"      {i}. {client}")

# Interceptor Configuration
interceptor_config = result['interceptorConfigurations'][0]
print(f"\n🛡️  Interceptor Configuration:")
print(f"   Lambda ARN: {interceptor_config['interceptor']['lambda']['arn']}")
print(f"   Interception Points: {interceptor_config['interceptionPoints']}")
print(f"   Pass Request Headers: {interceptor_config['inputConfiguration']['passRequestHeaders']}")
print(f"\n   ℹ️  The interceptor will:")
print(f"      • Examine JWT tokens in Authorization header")
print(f"      • Extract cognito:groups claim")
print(f"      • Enforce group-based permissions")
print(f"      • Allow/deny requests before reaching runtime")


インターセプター Gateway から呼び出せるように Lambda に権限を追加

In [ ]:
# Update Lambda permission with gateway ARN
import boto3

lambda_client = boto3.client('lambda', region_name=AWS_REGION)
function_name = f"{WORKSHOP_NAME}-interceptor-request"
gateway_arn = result['gatewayArn']

print(f"🔧 Updating Lambda permission with gateway ARN...")

# Remove old permission
try:
    lambda_client.remove_permission(
        FunctionName=function_name,
        StatementId='AllowGatewayInvoke'
    )
    print(f"   Removed old permission")
except:
    pass

# Add new permission with source ARN
lambda_client.add_permission(
    FunctionName=function_name,
    StatementId='AllowGatewayInvoke',
    Action='lambda:InvokeFunction',
    Principal='bedrock-agentcore.amazonaws.com',
    SourceArn=gateway_arn
)

print(f"✅ Lambda permission updated with source ARN")
print(f"   Gateway ARN: {gateway_arn}")

### ターゲット設定

**ターゲット詳細:**
- 名前: `mcp-remediation-target`
- タイプ: MCP サーバー
- エンドポイント: [ステップ 6 からのランタイムエンドポイント]

**認証情報プロバイダー:**
- タイプ: OAUTH
- プロバイダー: Cognito MCP プロバイダー

### ターゲットの作成

In [ ]:
# Get runtime endpoint from Lab 3A
runtime_arn = get_parameter(PARAMETER_PATHS['lab_03']['runtime_arn'])
runtime_endpoint = f"https://bedrock-agentcore.{AWS_REGION}.amazonaws.com/runtimes/{runtime_arn.replace(':', '%3A').replace('/', '%2F')}/invocations?qualifier=DEFAULT"
print (f'runtime endpoint: {runtime_endpoint}')
# Get OAuth provider ARN
oauth_provider_arn = get_parameter(PARAMETER_PATHS['lab_03']['oauth2_provider_arn'])

# Create target
response = agentcore_client.create_gateway_target(
    gatewayIdentifier=gateway_id,
    name="mcp-remediation-target",
    description="MCP server target for remediation agent with JWT auth",
    targetConfiguration={
        "mcp": {
            "mcpServer": {
                "endpoint": runtime_endpoint
            }
        }
    },
    credentialProviderConfigurations=[
        {
            "credentialProviderType": "OAUTH",
            "credentialProvider": {
                "oauthCredentialProvider": {
                    "providerArn": oauth_provider_arn,
                    "scopes": []
                }
            }
        }
    ]
)

target_id = response['targetId']
print(f"✅ Target Creating: {target_id}")
print(f"   Runtime: {runtime_arn}")


In [ ]:
# Wait for target to be READY
print("⏳ Waiting for target to be ready...")

target_info = agentcore_client.get_gateway_target(
          gatewayIdentifier=gateway_id,
          targetId=target_id
      )
time.sleep(5)
status = target_info.get('status', 'UNKNOWN')

if status == 'READY':
    print(f"✅ Target is READY")
    print(f"\n🎉 Gateway and target ready for testing!")
    print(f"   Gateway URL: {gateway_url}")
    print(f"   Target ID: {target_id}")
if status == 'FAILED' or status == 'SYNCHRONIZE_UNSUCCESSFUL':
    print(f"❌ Target in ERROR state: {target_info.get('statusReasons', 'No error message')}")

### ターゲット設定の確認

In [ ]:
# Display full target configuration
result = agentcore_client.get_gateway_target(
    gatewayIdentifier=gateway_id,
    targetId=target_id
)

print("📋 Target Configuration:")
print("=" * 70)
print(json.dumps(result, indent=2, default=str))
print("=" * 70)

# Highlight key configuration elements
print("\n🔑 Key Configuration Elements:")
print(f"   Target ID: {result['targetId']}")
print(f"   Target Name: {result['name']}")
print(f"   Status: {result['status']}")

# MCP Server Configuration
mcp_config = result['targetConfiguration']['mcp']['mcpServer']
print(f"\n🔌 MCP Server Configuration:")
print(f"   Endpoint: {mcp_config['endpoint'][:80]}...")
print(f"   ℹ️  Points to Lab 3A runtime (reused)")

# Credential Provider Configuration
cred_config = result['credentialProviderConfigurations'][0]
oauth_config = cred_config['credentialProvider']['oauthCredentialProvider']
print(f"\n🔐 Credential Provider Configuration:")
print(f"   Type: {cred_config['credentialProviderType']}")
print(f"   Provider ARN: {oauth_config['providerArn']}")
print(f"   ℹ️  Uses Cognito OAuth provider for machine-to-machine auth")

# Explain the flow
print(f"\n📊 Request Flow:")
print(f"   1. User sends request with JWT token → Gateway")
print(f"   2. Gateway validates JWT (Cognito OIDC)")
print(f"   3. Lambda interceptor checks permissions")
print(f"   4. If allowed → Gateway forwards to Target")
print(f"   5. Target uses OAuth to authenticate with Runtime")
print(f"   6. Runtime executes agent logic")
print(f"   7. Response flows back through Gateway")


---

# ステップ 6: MCP ランタイムのレビュー

## 目標
ゲートウェイ経由でアクセスされる修復エージェントランタイムをレビューする。

### ランタイム設定の確認

In [ ]:
# Verify runtime from Lab 3A is available
from lab_helpers.parameter_store import get_parameter
from lab_helpers.constants import PARAMETER_PATHS
from lab_helpers.config import AWS_REGION
import boto3

runtime_arn = get_parameter(PARAMETER_PATHS['lab_03']['runtime_arn'])
runtime_id = runtime_arn.split('/')[-1]

print(f"✅ Runtime from Lab 3A:")
print(f"   ARN: {runtime_arn}")
print(f"   ID: {runtime_id}")
print(f"   Region: {AWS_REGION}")

# Check runtime status using control plane client
agentcore_control = boto3.client('bedrock-agentcore-control', region_name=AWS_REGION)
try:
    runtime_info = agentcore_control.get_agent_runtime(agentRuntimeId=runtime_id)
    status = runtime_info['status']
    print(f"\n📊 Runtime Status: {status}")
    
    if status == 'READY':
        print(f"   ✅ Runtime is ready for gateway integration")
    elif status == 'FAILED' or status == 'SYNCHRONIZE_UNSUCCESSFUL':
        print(f"❌ Target in ERROR state: {target_info.get('statusReasons', 'No error message')}")
except Exception as e:
    print(f"   ❌ Error checking runtime: {e}")


---

# ステップ 7: Cognito トークンの再生成

## 目標
新しいゲートウェイで有効な、両方のペルソナ用の新しい JWT トークンを取得する。

### SRE ユーザートークン

In [ ]:
# Regenerate SRE token (in case Step 2 tokens expired)
sre_response = cognito.initiate_auth(
    ClientId=user_client_id,
    AuthFlow='USER_PASSWORD_AUTH',
    AuthParameters={'USERNAME': sre_username, 'PASSWORD': sre_password}
)

sre_token = sre_response['AuthenticationResult']['AccessToken']
sre_claims = decode_jwt(sre_token)

print(f"✅ Fresh SRE token generated")
print(f"   Username: {sre_username}")
print(f"   Groups: {sre_claims.get('cognito:groups', [])}")
print(f"   Token (first 50 chars): {sre_token[:50]}...")

### 承認者ユーザートークン

In [ ]:
# Regenerate Approver token (in case Step 2 tokens expired)
approver_response = cognito.initiate_auth(
    ClientId=user_client_id,
    AuthFlow='USER_PASSWORD_AUTH',
    AuthParameters={'USERNAME': approver_username, 'PASSWORD': approver_password}
)

approver_token = approver_response['AuthenticationResult']['AccessToken']
approver_claims = decode_jwt(approver_token)

print(f"✅ Fresh Approver token generated")
print(f"   Username: {approver_username}")
print(f"   Groups: {approver_claims.get('cognito:groups', [])}")
print(f"   Token (first 50 chars): {approver_token[:50]}...")

## メモリからの診断情報でコンテキストを充実させる

診断情報でコンテキストを充実させるために、キュレートされたメモリから追加情報を取得しましょう。

In [ ]:
agent_memory_client = boto3.client("bedrock-agentcore", region_name=AWS_REGION)

memory_id = get_parameter(PARAMETER_PATHS['memory']['memory_id'])
memory_session_id = get_parameter(PARAMETER_PATHS['memory']['default_session_id'])

print(memory_id)
print(memory_session_id)
actor_id='diagnostics_agent'

  

#list events added to agent memory, to confirm successful write
params = {
                "memoryId": memory_id,
                "actorId": actor_id,
                "sessionId": memory_session_id,
                "includePayloads": True
            }
# Get all messages
response = agent_memory_client.list_events(**params)
additional_context=""
for event in response.get("events", []):
    payload = event.get('payload', [])
    for i, item in enumerate(payload):
        if 'conversational' in item:
            text = item['conversational']['content']['text']
            additional_context+=text
additional_context

---

# ステップ 8: SRE が計画作成をシミュレート

## 目標
SRE ユーザーが修復計画を正常に作成できることを示す。

## シナリオ
SRE が問題を検出し、それを修復するための計画を作成する。

### 計画作成の実行

In [ ]:
# Connect to gateway with SRE token
from lab_helpers.lab_03.mcp_client import MCPClient

gateway_url = get_parameter(PARAMETER_PATHS['lab_03b']['gateway_url'])

print(f"🔗 Connecting to gateway as SRE user...")
print(f"   Gateway: {gateway_url}")
print(f"   User: {sre_username}")
print(f"   Groups: {sre_claims.get('cognito:groups', [])}")

sre_client = MCPClient(gateway_url, sre_token)
sre_client.initialize()

print(f"\n✅ Connected to gateway")

In [ ]:
# List available tools
print(f"🔧 Listing available tools...\n")
tools = sre_client.list_tools()

print(f"📋 Available tools ({len(tools)}):")
for tool in tools:
    print(f"   • {tool['name']}")
    if 'description' in tool:
        print(tool)
        desc = tool['description']
        print(f"     {desc[:80]}..." if len(desc) > 80 else f"     {desc}")

In [ ]:
# Invoke remediation planning tool
start_time = datetime.now()
print(f"\n🎯 Invoking generate_remediation_plan tool...\n")
try:
    result = sre_client.call_tool(
        tool_name='mcp-remediation-target___infrastructure_agent',
        arguments={
            'remediation_query': f"""I need help with infrastructure remediation for our CRM application. We're experiencing: {additional_context} """,
            'action_type': 'only_plan'
        }
    )
except Exception as e:
    print(f"❌ Error: {e}")
analysis_time = (datetime.now() - start_time).total_seconds()
print(f"Analysis Time: {analysis_time:.2f} seconds")
print(f"✅ Tool invocation successful!")
print(f"\n📋 Remediation Plan:")
print(f"{result[:500]}..." if len(result) > 500 else result)

### 成功の確認

In [ ]:
print(f"\n✅ SRE User Successfully:")
print(f"   1. Connected to gateway with JWT token")
print(f"   2. Listed available tools")
print(f"   3. Invoked generate_remediation_plan tool")
print(f"   4. Received remediation plan")
print(f"\n🔒 Interceptor allowed this operation because:")
print(f"   • User is in 'sre' group")
print(f"   • Tool 'generate_remediation_plan' is allowed for SRE users")

### 期待される結果
✅ SRE トークンで計画が正常に作成

---

# ステップ 9: SRE が実行を試行（アクセス拒否）

## 目標
SRE ユーザーが修復を実行できないことをデモンストレーション—インターセプターがブロックする。

## シナリオ
SRE が作成した計画を実行しようとする。

### 実行の試行

In [ ]:
# Invoke remediation planning tool
print(f"\n🎯 Invoking execute_remediation_step tool...\n")

result = sre_client.call_tool(
    tool_name='mcp-remediation-target___infrastructure_agent',
    arguments={
            "remediation_query": f"""
            Help me change the read and write capacity to on-demand for the DynamoDb tables in my CRM application, based on this diagnostic information: {additional_context}
            """,
            "action_type": "only_execute"
        }
)

### アクセス拒否の分析

In [ ]:
# Check Lambda logs to see the interceptor rejection
# Since it may take few minutes for logs to reach cloudwatch, please wait and retry
import boto3
from datetime import datetime

logs = boto3.client('logs', region_name=AWS_REGION)
log_group = '/aws/lambda/aiml301_sre_agentcore-interceptor-request'

streams = logs.describe_log_streams(
    logGroupName=log_group,
    orderBy='LastEventTime',
    descending=True,
    limit=1
)

print("📋 Interceptor Lambda Logs (Last 20 lines):\n")
stream = streams['logStreams'][0]
events = logs.get_log_events(
    logGroupName=log_group,
    logStreamName=stream['logStreamName'],
    limit=30,
    startFromHead=False
)

for event in events['events'][-20:]:
    msg = event['message'].strip()
    if any(x in msg for x in ['Tool call', 'User groups', 'not authorized', 'Denying']):
        print(msg)

print("\n✅ The interceptor blocked the execute_remediation_step call for SRE user")

### 期待される結果
❌ アクセス拒否 - インターセプターの拒否が表示される

---

# ステップ 10: 承認者が実行と検証

## 目標
承認者ユーザーが修復を実行および検証できることを示す。

### パート A: 承認者が修復を実行

In [ ]:
approver_client = MCPClient(gateway_url, approver_token)
approver_client.initialize()

In [ ]:
# Invoke remediation planning tool
print(f"\n🎯 Invoking execute_remediation_step tool...\n")
start_time = time.time()
try:
    result = approver_client.call_tool(
        tool_name='mcp-remediation-target___infrastructure_agent',
        arguments={
            "remediation_query": f"""
            This is a test invocation to validate access to infrastructure_agent and tool. Please do a health check and confirm your status.
            """,
            "action_type": "only_execute"
        }
    )
except Exception as e:
    print(f"❌ Error: {e}")
end_time = time.time()
print(f"  ⏰ Total time taken: {end_time - start_time:.2f} seconds")

### パート B: 承認者が修正を検証

### 期待される結果
✅ 承認者トークンで修復が実行および検証

---

# クリーンアップ

## 目標
Lab 3A の基本インフラストラクチャを保持しながら、Lab 3B リソースを削除する。

**削除されるリソース:**
- JWT 認証を持つゲートウェイ
- Lambda インターセプター関数
- Lambda 実行ロール

**保持されるリソース:**
- AgentCore ランタイム（Lab 3A から）
- Cognito ユーザープールとユーザー
- OAuth2 認証情報プロバイダー
- Parameter Store エントリ

In [ ]:
from lab_helpers.lab_03 import cleanup_lab_03b
from lab_helpers.config import AWS_REGION

#cleanup_lab_03b(region_name=AWS_REGION, verbose=True)

---

# 参考資料
- Lab 3A: 修復エージェントの基礎
- Cognito JWT 設定
- AgentCore Gateway ドキュメント
- Lambda インターセプターパターン